In [1]:
import torch 
import torch.nn.functional as F

In [4]:
t1 = torch.randn(128, 500, 300)
t2 = torch.randn(128)

t1 * t2.unsqueeze(1).unsqueeze(2)

tensor([[[-1.8710e+00, -1.0877e+00, -2.4461e-01,  ...,  9.5277e-01,
           2.4916e+00, -1.1813e+00],
         [ 4.8908e-01, -1.5438e+00,  1.1630e+00,  ..., -1.0864e-01,
          -1.3692e+00, -3.5051e-01],
         [-9.7979e-01, -5.8967e-01,  5.4338e-02,  ..., -9.8899e-01,
          -4.6536e-01, -3.4412e-02],
         ...,
         [-1.9306e-01, -6.7972e-01, -3.6876e-01,  ..., -6.4601e-01,
           8.7687e-02,  2.8251e-01],
         [ 8.1259e-01, -8.3520e-02,  7.6828e-01,  ...,  1.3538e-02,
          -1.0519e+00,  1.1976e-01],
         [ 3.7449e-01, -2.5942e-01,  1.0785e+00,  ..., -1.2454e+00,
          -3.8538e-01, -1.0500e+00]],

        [[-4.3624e+00,  2.5205e+00,  1.7421e-01,  ..., -3.1031e+00,
          -5.6478e-01,  2.7436e+00],
         [-3.7902e-01, -5.8769e-01,  7.4534e-02,  ..., -7.7646e-01,
          -1.7398e+00, -1.3992e+00],
         [-7.2495e-01,  1.9109e+00,  8.8269e-01,  ..., -2.1507e+00,
          -2.5480e+00,  1.7521e+00],
         ...,
         [-9.8692e-01, -2

In [62]:
aspects_projection = []

for i in range(5):
    result = torch.matmul(t1, t2[i, :, :])
    aspects_projection.append(result)

torch.stack(aspects_projection, dim=2).shape

torch.Size([128, 500, 5, 32])

In [53]:
F.conv2d(t1, t2, padding=1)

RuntimeError: expected stride to be a single integer value or a list of 1 values to match the convolution dimensions, but got stride=[5, 1]

In [64]:
result = torch.matmul(t1, t2)

In [65]:
result.shape

torch.Size([128, 5, 500, 32])

In [8]:
result.shape

torch.Size([])

In [1]:
import numpy as np

In [9]:
topics = np.ndarray(shape=(10, 32))
ids = np.array(range(0, 5))

In [16]:
topics[ids+3]

array([[ 1.28457068e-322,              nan,  0.00000000e+000,
         4.94065646e-324,  3.59679790e-321,  4.44659081e-323,
         6.94458961e-310,  4.66608425e-310,  4.94065646e-324,
         1.28457068e-322,              nan,  6.94459033e-310,
         4.94065646e-324,  3.95252517e-321,  1.48219694e-323,
         6.94458961e-310,  4.66608425e-310,  0.00000000e+000,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         4.94065646e-324,  0.00000000e+000,  6.94458961e-310,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000,  0.00000000e+000],
       [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000,  6.94458961e-310,  6.94458961e-310,
         3.95252517e-321,  9.88131292e-324,  6.94458961e-310,
         4.66608425e-310,              nan,  1.28457068e-322,
                     nan,